# 合并数据集：Concat与Append操作

In [1]:
import numpy as np
import pandas as pd

In [2]:
def make_df(cols, ind):
    """一个简单的DataFrame"""
    data = {c: [str(c) + str(i) for i in ind] for c in cols}
    return pd.DataFrame(data, ind)

make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In [3]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## 1. 知识回顾：NumPy数组的合并

合并 `Series` 与 `DataFrame` 与合并 `NumPy` 数组基本相同，后者通过 2.2 节中介绍的 `np.concatenate` 函数即可完成。  
你可以用这个函数将两个或两个以上的数组合并成一个数组。

In [4]:
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [5]:
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

## 2. 通过pd.concat实现简易合并

`Pandas` 有一个 `pd.concat()` 函数与 `np.concatenate` 语法类似，但是配置参数更多，功能也更强大。

```python
# Signature in Pandas v0.18
pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
```

In [6]:
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])  # pd.concat() 可以简单地合并一维的 Series 或 DataFrame 对象，与 np.concatenate() 合并数组一样

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [7]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
# print(df1); print(df2); print(pd.concat([df1, df2]))
display('df1', 'df2', 'pd.concat([df1, df2])')  # 也可以用来合并高维数据

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [8]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
display('df3', 'df4', "pd.concat([df3, df4], axis='columns')")  # 也可以使用 axis=1，效果是一样的。但是用 axis='columns' 会更直观。

df3
    A   B
0  A0  B0
1  A1  B1

df4
    C   D
0  C0  D0
1  C1  D1

pd.concat([df3, df4], axis='columns')
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1

### 2.1. 索引重复

`np.concatenate` 与 `pd.concat` 最主要的差异之一就是 `Pandas` 在合并时会保留索引，即使索引是重复的。

In [9]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # 复制索引
display('x', 'y', 'pd.concat([x, y])')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
0,A2,B2
1,A3,B3


你会发现结果中的索引是重复的。虽然 `DataFrame` 允许这么做，但结果并不是我们想要的。  
`pd.concat()` 提供了一些解决这个问题的方法。

#### 2.1.1. 捕捉索引重复的错误

如果你想要检测 `pd.concat()` 合并的结果中是否出现了重复的索引，可以设置 `verify_integrity` 参数。  
将参数设置为 `True`，合并时若有索引重复就会触发异常。

In [10]:
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Int64Index([0, 1], dtype='int64')


#### 2.1.2. 忽略索引

有时索引无关紧要，那么合并时就可以忽略它们，可以通过设置 `ignore_index` 参数来实现。  
如果将参数设置为 `True`，那么合并时将会创建一个新的整数索引。

In [11]:
display('x', 'y', 'pd.concat([x, y], ignore_index=True)')

,A,B
0,A0,B0
1,A1,B1
,A,B
0,A2,B2
1,A3,B3
,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


#### 2.1.3. 增加多级索引

另一种处理索引重复的方法是通过 `keys` 参数为数据源设置多级索引标签，这样结果数据就会带上多级索引。

In [12]:
display('x', 'y', "pd.concat([x, y], keys=['x', 'y'])")

x
    A   B
0  A0  B0
1  A1  B1

y
    A   B
0  A2  B2
1  A3  B3

pd.concat([x, y], keys=['x', 'y'])
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3

### 2.2. 类似join的合并

In [13]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
display('df5', 'df6', 'pd.concat([df5, df6])')  # 默认情况下，某个位置上缺失的数据会用 NaN 表示。默认的合并方式是对所有输入列进行并集合并（join='outer'）

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6])
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4

In [14]:
display('df5', 'df6', "pd.concat([df5, df6], join='inner')")  # 用 join='inner' 实现对输入列的交集合并

df5
    A   B   C
1  A1  B1  C1
2  A2  B2  C2

df6
    B   C   D
3  B3  C3  D3
4  B4  C4  D4

pd.concat([df5, df6], join='inner')
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4

In [15]:
# pandas 1.0版本以后 不推荐使用join_axes参数，会报错 TypeError: concat() got an unexpected keyword argument ‘join_axes’；pandas 1.0之前仍可使用
# display('df5', 'df6', "pd.concat([df5, df6], join_axes=[df5.columns])")

# 预期效果
"""
   A    B   C
1  A1   B1  C1
2  A2   B2  C2
3  NaN  B3  C3
4  NaN  B4  C4
"""

'\n   A    B   C\n1  A1   B1  C1\n2  A2   B2  C2\n3  NaN  B3  C3\n4  NaN  B4  C4\n'

### 2.3. append()方法

因为直接进行数组合并的需求非常普遍，所以 `Series` 和 `DataFrame` 对象都支持 `append` 方法，让你通过最少的代码实现合并功能。

In [16]:
display('df1', 'df2', 'df1.append(df2)')

,A,B
1,A1,B1
2,A2,B2
,A,B
3,A3,B3
4,A4,B4
,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


需要注意的是，与 `Python` 列表中的 `append()` 和 `extend()` 方法不同，`Pandas` 的 `append()` 不直接更新原有对象的值，而是为合并后的数据创建一个新对象。  
因此，它不能被称之为一个非常高效的解决方案，因为每次合并都需要重新创建索引和数据缓存。  
总之，如果你需要进行多个 `append` 操作，还是建议先创建一个 `DataFrame` 列表，然后用 `concat()` 函数一次性解决所有合并任务。